In [1]:
import pandas as pd
import numpy as np 

In [2]:
train = pd.read_csv('train.csv')
macro = pd.read_csv('macro.csv')

# General Characteristics 

In [3]:
train.shape

(30471, 292)

In [3]:
train.isna().sum().sort_values()[0:50]

id                            0
market_count_500              0
green_part_1000               0
prom_part_1000                0
office_count_1000             0
office_sqm_1000               0
trc_count_1000                0
trc_sqm_1000                  0
cafe_count_1000               0
cafe_count_1000_na_price      0
cafe_count_1000_price_500     0
cafe_count_1000_price_1000    0
cafe_count_1000_price_1500    0
cafe_count_1000_price_2500    0
cafe_count_1000_price_4000    0
sport_count_500               0
cafe_count_1000_price_high    0
church_count_1000             0
mosque_count_1000             0
leisure_count_1000            0
sport_count_1000              0
market_count_1000             0
green_part_1500               0
prom_part_1500                0
office_count_1500             0
office_sqm_1500               0
trc_count_1500                0
trc_sqm_1500                  0
cafe_count_1500               0
cafe_count_1500_na_price      0
cafe_count_1500_price_500     0
big_chur

Features: actual home and the location (more towards the city center, it would be more expensive, judging from personal experience

In [1]:
features_train = ["full_sq","metro_min_walk","big_market_km",
                  "workplaces_km","university_km","cafe_count_1000","shopping_centers_km","office_km",
                  "big_church_km","school_education_centers_top_20_raion","build_count_after_1995",
                  "cafe_count_1500_price_500","market_count_500","oil_chemistry_km","railroad_km","ts_km",
                  "young_all","work_male","work_female","ekder_all","build_count_mix",
                  "build_count_1971-1995","build_count_1946-1970","build_count_1921-1945","build_count_before_1920"]

In [5]:
features_subset = train[features_train]

In [6]:
features_subset.isna().sum()

full_sq                                     0
metro_min_walk                             25
big_market_km                               0
big_market_km                               0
workplaces_km                               0
university_km                               0
cafe_count_1000                             0
shopping_centers_km                         0
office_km                                   0
big_church_km                               0
school_education_centers_top_20_raion       0
build_count_after_1995                   4991
cafe_count_1500_price_500                   0
market_count_500                            0
oil_chemistry_km                            0
railroad_km                                 0
ts_km                                       0
young_all                                   0
work_male                                   0
work_female                                 0
ekder_all                                   0
build_count_mix                   

In [7]:
features_subset[features_subset["build_count_mix"].isna()][["build_count_mix","build_count_1971-1995","build_count_1946-1970"]].head()

,build_count_mix,build_count_1971-1995,build_count_1946-1970
2042,NaN,NaN,NaN
2252,NaN,NaN,NaN
2263,NaN,NaN,NaN
2319,NaN,NaN,NaN
2715,NaN,NaN,NaN


The building counts for each of the type of buildng is missing for the all the same rows.

In [8]:
# fill metro with mean
features_subset["metro_min_walk"] = features_subset["metro_min_walk"].fillna(features_subset["metro_min_walk"].mean())

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Random forest imputing building counts with all the other features but not the target

In [10]:
import missingpy

In [11]:
rf_imp = missingpy.MissForest(criterion = "mse")

In [12]:
filled_features = rf_imp.fit_transform(features_subset)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4


In [13]:
data_imp = pd.DataFrame(filled_features, columns = features_subset.columns)

In [14]:
data_imp["target"] = train["price_doc"]

In [15]:
data_imp["timestamp"] = train["timestamp"]

In [16]:
from sklearn.linear_model import Lasso

In [17]:
data_imp.columns

Index(['full_sq', 'metro_min_walk', 'big_market_km', 'big_market_km',
       'workplaces_km', 'university_km', 'cafe_count_1000',
       'shopping_centers_km', 'office_km', 'big_church_km',
       'school_education_centers_top_20_raion', 'build_count_after_1995',
       'cafe_count_1500_price_500', 'market_count_500', 'oil_chemistry_km',
       'railroad_km', 'ts_km', 'young_all', 'work_male', 'work_female',
       'ekder_all', 'build_count_mix', 'build_count_1971-1995',
       'build_count_1946-1970', 'build_count_1921-1945',
       'build_count_before_1920', 'target', 'timestamp'],
      dtype='object')

In [18]:
l_feat = Lasso(alpha = .4)
l_feat.fit(data_imp.drop(["timestamp","target"], axis = 1),data_imp["target"])
coefs = l_feat.coef_

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.3678011195817997e+17, tolerance: 69622317794293.57
  positive)


In [19]:
selected_features = []
cols = data_imp.drop(["timestamp","target"], axis = 1).columns
for i in range(len(coefs)):
    if coefs[i] != 0:
        selected_features.append(cols[i])
selected_features

['full_sq',
 'metro_min_walk',
 'big_market_km',
 'big_market_km',
 'workplaces_km',
 'university_km',
 'cafe_count_1000',
 'shopping_centers_km',
 'office_km',
 'big_church_km',
 'school_education_centers_top_20_raion',
 'build_count_after_1995',
 'cafe_count_1500_price_500',
 'market_count_500',
 'oil_chemistry_km',
 'railroad_km',
 'ts_km',
 'young_all',
 'work_male',
 'work_female',
 'ekder_all',
 'build_count_mix',
 'build_count_1971-1995',
 'build_count_1946-1970',
 'build_count_1921-1945',
 'build_count_before_1920']

In [20]:
subset_feature = data_imp[selected_features]

In [21]:
subset_feature["target"] = data_imp["target"]
subset_feature["timestamp"] = data_imp["timestamp"]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# Macro Economic Characteristics

In [22]:
macro.iloc[2484,:].head()

timestamp            50.2
oil_urals           45.71
gdp_quart            38.4
gdp_quart_growth    29.78
cpi                   NaN
Name: 2484, dtype: object

one of the timestamps is 50.2 so we will remove this row

In [23]:
macro.drop(index = 2484, inplace = True)

In [24]:
macro["timestamp"] = pd.to_datetime(macro["timestamp"])

In [25]:
macro.shape

(2484, 100)

In [26]:
macro.isna().sum().sort_values()

timestamp                                    0
oil_urals                                    0
fixed_basket                                 0
mortgage_rate                                0
mortgage_value                               0
deposits_value                               0
average_provision_of_build_contract          0
gdp_annual                                   0
gdp_annual_growth                            0
micex_cbi_tr                                 1
usdrub                                       3
eurrub                                       3
brent                                        3
micex                                       10
rts                                         10
micex_rgbi_tr                               10
balance_trade                               31
ppi                                         31
deposits_growth                             31
cpi                                         31
balance_trade_growth                        90
gdp_quart    

In [28]:
macro_train = ["timestamp","unemployment","salary","construction_value","rent_price_1room_eco","rent_price_1room_bus",
               "cpi","childbirth","marriages_per_1000_cap","eurrub","deposits_value"]

In [29]:
len(macro_train)

11

In [30]:
macro_subset = macro[macro_train]

In [31]:
macro_subset.isna().sum()

timestamp                   0
unemployment              293
salary                    293
construction_value        658
rent_price_1room_eco      273
rent_price_1room_bus      273
cpi                        31
childbirth                293
marriages_per_1000_cap    658
eurrub                      3
deposits_value              0
dtype: int64

In [32]:
macro_subset.set_index("timestamp", inplace = True)

In [33]:
macro_subset.dtypes

unemployment              float64
salary                    float64
construction_value        float64
rent_price_1room_eco      float64
rent_price_1room_bus      float64
cpi                       float64
childbirth                float64
marriages_per_1000_cap    float64
eurrub                    float64
deposits_value            float64
dtype: object

Imputing NA values for time series: https://medium.com/@drnesr/filling-gaps-of-a-time-series-using-python-d4bfddd8c460

In [34]:
macro_filled = macro_subset.interpolate(method = "time")

In [35]:
macro_filled.isna().sum()

unemployment                0
salary                      0
construction_value          0
rent_price_1room_eco      273
rent_price_1room_bus      273
cpi                        31
childbirth                  0
marriages_per_1000_cap      0
eurrub                      3
deposits_value              0
dtype: int64

Some NA values remain mostly due to them being at the beginning of the dataset, so there is no trend to begin interpolating on. As such, we will simply backfill them.

In [36]:
macro_filled.fillna(method = "bfill", inplace = True)

In [37]:
macro_filled.isna().sum()

unemployment              0
salary                    0
construction_value        0
rent_price_1room_eco      0
rent_price_1room_bus      0
cpi                       0
childbirth                0
marriages_per_1000_cap    0
eurrub                    0
deposits_value            0
dtype: int64

In [38]:
macro_filled.reset_index(inplace = True)

In [39]:
macro_filled.head()

,timestamp,unemployment,salary,construction_value,rent_price_1room_eco,rent_price_1room_bus,cpi,childbirth,marriages_per_1000_cap,eurrub,deposits_value
0,2010-01-01,0.018,38410.5,553874.9,27.58,45.44,315.1,10.7,8.0,43.4054,7484970.0
1,2010-01-02,0.018,38410.5,553874.9,27.58,45.44,315.1,10.7,8.0,43.4054,7484970.0
2,2010-01-03,0.018,38410.5,553874.9,27.58,45.44,315.1,10.7,8.0,43.4054,7484970.0
3,2010-01-04,0.018,38410.5,553874.9,27.58,45.44,315.1,10.7,8.0,43.4054,7484970.0
4,2010-01-05,0.018,38410.5,553874.9,27.58,45.44,315.1,10.7,8.0,42.9600,7484970.0


In [40]:
subset_feature["timestamp"] = pd.to_datetime(subset_feature["timestamp"])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [41]:
merged = subset_feature.merge(macro_filled, on='timestamp', how='left')
print(merged.shape)
merged.head()

(30471, 40)


,full_sq,metro_min_walk,big_market_km,big_market_km,big_market_km,big_market_km,workplaces_km,university_km,cafe_count_1000,shopping_centers_km,...,unemployment,salary,construction_value,rent_price_1room_eco,rent_price_1room_bus,cpi,childbirth,marriages_per_1000_cap,eurrub,deposits_value
0,43.0,13.575119,10.814172,10.814172,10.814172,10.814172,0.884350,6.715026,19.0,0.648488,...,0.014,44898.7,549075.8,29.07,47.85,354.0,10.8,8.5,41.7681,10618898.0
1,34.0,7.620630,6.910568,6.910568,6.910568,6.910568,0.686252,1.852560,13.0,0.519311,...,0.014,44898.7,549075.8,29.07,47.85,354.0,10.8,8.5,41.7537,10618898.0
2,43.0,17.351515,5.752368,5.752368,5.752368,5.752368,1.510089,0.841254,9.0,1.486533,...,0.014,44898.7,549075.8,29.07,47.85,354.0,10.8,8.5,41.7114,10618898.0
3,89.0,11.565624,27.892717,27.892717,27.892717,27.892717,0.622272,10.903161,12.0,0.599914,...,0.014,44898.7,549075.8,30.63,51.15,353.2,10.8,8.5,41.4014,10720911.0
4,77.0,8.266305,9.155057,9.155057,9.155057,9.155057,0.892668,0.991683,153.0,0.429052,...,0.014,44898.7,549075.8,30.63,51.15,353.2,10.8,8.5,41.5902,10920215.0


In [42]:
merged.to_csv("final_NN_feat.csv")

In [50]:
train_labels = pd.DataFrame(train['price_doc'])

In [53]:
train_labels.iloc[1:,].to_csv('training_labels.csv')